In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import torch

## Name generation using bigrams and statistics
Below is a code that reads the input file with names and transforms it into a histogram of bigrams. Bigram is a pair of letters that come one after the other.
We calculate the probability of each bigram (i.e. if there's a letter "a", what are the chances for letters "b", "c", "d" coming next), and we "roll the dice", until we predict an end of sequence.

In [6]:
# Read the data set
words = open('./names.txt', 'r').read().splitlines()
print(f"Data set contains {len(words)} words")

Data set contains 32034 words


In [7]:
# Storing the information in the 2D array
# The 2D array rows represent a first letter in a bigram, and the column represents a second letter in a bigram.
# A value in (x,y) represents number of occurences of such bigram in the dataset.

bigrams = torch.zeros([27, 27], dtype=torch.int32)

In [8]:
# Define char to int and int to char mappings
all_chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i, s in enumerate(all_chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}

In [9]:
# Count bigram (2 letter pairs) occurences in the dataset. 
# At first, <S> was used as a special start character and <E> as special end character, but Andrej changed both to just '.' for efficiency.
for word in words:
    chars = ['.'] + list(word) + ['.']
    if len(chars) == 2: 
        raise Exception("no word")
    for ch1, ch2 in zip(chars, chars[1:]):
        bigrams[stoi[ch1], stoi[ch2]] += 1

Exception: no word

In [ ]:
plt.figure(figsize=(14,14))
plt.imshow(bigrams, cmap='Blues')
for i in range(len(bigrams)):
    for j in range(len(bigrams[0])):
        str = itos[i] + itos[j]
        plt.text(j, i, str, ha="center", va="bottom", color="grey")
        plt.text(j, i, bigrams[i, j].item(), ha="center", va="top", color="grey")
plt.axis('off')

In [ ]:
# Playground with Pytorch generators and dividing tensors with scalar value
g = torch.Generator().manual_seed(2147483647)
p = torch.rand(3, generator=g)
p = p / p.sum()
p

In [ ]:
# Torch.multinomial
# You give me probabilities, I give you integres which are sampled according to the probability distribution.
# So if I input a tensor to this function like this:
# probabilities = torch.tensor([0.1, 0.2, 0.3, 0.4])
# ... and I call the method like this:
# samples = probabilities.multinomial(num_samples=5, replacement=True)
# ... I can get a following result:
# tensor([2, 3, 1, 3, 0])
# It means "I have conducted 5 experiments and randomly generated this sequence of numbers according to your prob distribution. I have drawn value
# from index 2 first, then value from index 3, then 1, etc."
torch.multinomial(p, num_samples=20, replacement=True, generator=g)

In [ ]:
# Take first row of bigrams (start/end special character)
p = bigrams[0].float()
# Normalize it - each cell will now have a probablity of occurence and they will all sum to one
p = p / p.sum()
g = torch.Generator().manual_seed(2147483647)
# Generate the next character according to p distribution
idx = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
idx

In [ ]:
# Model smoothing - explained below
P = (bigrams+1).float()
# Normalize the bigram array with probability distributions.
# "1" means collapse the 1-st dimension (colums)
# TODO write here about the rules of broadcasting - how do you align dimensions and validate that the division works, etc.
P /= P.sum(1, keepdim=True)

In [ ]:
# Token generation logic
row = 0 # Start at index 0 - start token
word = ''
while True:
    selected = torch.multinomial(P[row], num_samples=1, replacement=True, generator=g).item()
    print(f'selected {itos[selected]} with prob {P[row][selected]}')
    row = selected
    if row == 0: break # 'End' character
    word += itos[row]
word

## Log likelihood
One technique to check the quality of the model is to run it against the training data.
We want to look at the probability that the code assigns to each of the bigrams.
We can compare it against the evenly distributed probablility (if everything is equally likely), as if language was just random. If it's higher than that, that's a good thing, it means that our algorithm learned something from the data.
The problem to devise such metric is called "maximum likelihood estimation".

Likelihood is calculated as a product of the probabilities. But becomes a very small number, so it's impracical. It's better to work with log likelihood function instead. 

### Smoothing
Trick to add fake counts of data. This is to avoid infinite loss (so inifite negative log likelihood) because log(0) = -inf


In [ ]:
# Function calculating negative log likelihood of the input data. This is a commonly used loss function for classifiers.
# Goal: We want to maximize the likelihood of the data w.r.t. to model parameters.
# -> This is equivalent to maximizing the log likelihood, as log is monotonic.
# -> Equivalent to minimizing the negative log likelihood.
# -> Equivalent to minimizing the negative average log likelihood.
def calculate_negative_log_likelihood(input, verbose=False):
    loglikelihood = 0.0
    n = 0
    for w in input:
        chars = ['.'] + list(w) + ['.']
        for ch1, ch2 in zip(chars, chars[1:]): 
            prob = P[stoi[ch1], stoi[ch2]]
            logprob = torch.log(prob)
            # log(a*b*c) = log(a) + log(b) + log(c)
            loglikelihood += logprob
            n += 1
            if verbose: print(f'{ch1}{ch2}, prob: {prob:.4f}, logprob: {logprob:.4f}')
    # Negate - we want the cost to be higher as the value of log likelihood becomes more negative.
    nll = -loglikelihood 
    return nll, n

nll, n = calculate_negative_log_likelihood(words)
print(f'average loglikelihood: {nll/n}')

## Bigrams using neural nets
The following is re-doing the same exercise but this time using neural networks, cost function and gradient descent optimization algorithm. 

In [ ]:
# Re-creating the training set, this time splitting into xs and ys instead of the occurences 2D array.
# Creating the training set below.
xs, ys = [], []

for word in words:
    chars = ['.'] + list(word) + ['.']
    for ch1, ch2 in zip(chars, chars[1:]):
        xs.append(stoi[ch1])
        ys.append(stoi[ch2])

# torch.tensor vs torch.Tensor - the first one infers the dtype automatically, whereas the second one uses float32.
# training set: x[i] input should have y[i] output
xs = torch.tensor(xs)
ys = torch.tensor(ys)


## One-hot encoding 
The process of one-hot encoding involves creating a binary vector for each unique category in the dataset, with the size of the vector equal to the number of unique categories. Each vector has only one element set to 1, representing the presence of the corresponding category, while all other elements are set to 0.

For us, this means 1x27 tensor, with a "1" at stoi[character]-th position, and "0"s elsewhere.

In [ ]:
import torch.nn.functional as F

xenc = F.one_hot(xs, num_classes=27).float() # by default it's torch.int64
print(f'one-hot X shape: {xenc.shape}, datatype: {xenc.dtype}')

## Modelling neural network and forward() as matrices
We will model inputs as Nx27 matrix, and neural network layer as 27xM matrix.
In the input matrix, each row is a single example from the training set, encoded in the one-hot vector.
In the NN matrix, each **column** is a single neuron. Each row corresponds to weights for given input i_n
So, if we have 10 examples in the training set, the input is a 5x27 matrix.
If we have a 'network' with just one neuron, NN matrix will be 27x1.
The resulting matrix multiplication output will perform dot product of all the weights and inputs in the neuron, and will be 5x1 - the output per each training set example. Then we will only need to add neuron bias and non-linearity like tanh and voila, done.

In [ ]:
# Fills the tensor with normal distribution - i.e. initialize the weights to random according to it.
W = torch.randn([27, 1]) # 27 weights for a single neuron
xenc[:5] @ W # 5x27 @ 27x1 = 5x1

W = torch.randn([27, 27]) # 27 neurons, each connected to 27 inputs.
forward = (xenc[:5] @ W)
print(forward.shape)
print(forward[3, 13]) # What was the output of 13-th neuron on the 3-rd input?

## Softmax
Popular NN layer, that takes the output of the NN layer - some numbers aka logits, and transforms them into probabilty distribution.
It exponaties them and normalizes. It outputs probability distribution - numbers that are positive and sum to one.

![img](.//softmax_function.svg)

For our example, it will allow to reference the output of the NN to the original approach with the statistical occurence counting.

In [ ]:
# We will make the neuron network return **log counts**
# This is so that we can have a better intuition on what the NN returns. 
# Right now we can't have an intuition on the inner representation of neuron's weights.

logits = xenc @ W # log-counts
# This and a second line is a softmax!
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True) # WARN - if I don't do keepdim=True, this returns a wrong result! This is due to Pytorch broadcasting.
print(f'prob.shape: {prob.shape}, prob first row: {prob[1]}, sum of the first row (should be one, those are probs): {prob[1].sum()}')


In [ ]:
# After the transformations above, we will interpret the output of each row of the resulting output matrix as distribution of probabilities.
# In other words, neuron 0's output is a probability that the next character is start/stop, neuron 1's out is 'a', etc.

def calculate_negative_log_likelihood_nn(xs, ys):
    nlls = torch.zeros(len(xs))
    for i in range(len(xs)):
        x = xs[i].item()
        y = ys[i].item()
        p = prob[i, y] # for i-th example, how likely does the model think is the output from the training set?
        nll = -p.log()
        nlls[i] = nll
        print(f'characters: {itos[x]},{itos[y]}, model ({y}-th neuron output) is {p}. Nll: {nll}')
    return nlls

nlls = calculate_negative_log_likelihood_nn(xs[:5],ys[:5])
print(f'average negative log likelihood: {nlls.sum()/len(nlls)}')

## Neural net training!
We are doing the same thing as with micrograd. For the training set, we need to:
1. Do forward pass - run the NN on all the inputs.
2. Calculate the loss - for our case negative log likelihood 
3. Do the backward pass - calculate the gradients.
4. Update the weigths (W matrix) with the new values of gradient.

In [ ]:
g = torch.Generator().manual_seed(2147483647)
# Initialize 27 neurons, each with 27 inputs for one-hot encoded characters.
W = torch.randn((27, 27), generator=g, requires_grad=True)


In [ ]:
# xenc - our input for training data, one-hot encoded first characters of the bigrams
# y - out output of the training data, for xenc[i] this is the expected next letter in the bigram.
def train(iters=10):
    for i in range(iters):
        print(f'iteration {i}')
        # Forward pass
        logits = xenc @ W
        counts = logits.exp()
        softmax = counts/counts.sum(1, keepdim=True)
        # Calculate loss - NLL
        # The softmax is an output of NN for all inputs. So the formula below will return for i-th row the corresponding y[i]-th value.
        # This column is the one we care about, as this is a training example. E.g. for 10th training example c->d so for 10th row we should 
        # take 4 column (d's index).
        probs_for_examples = softmax[torch.arange(xenc.shape[0]), ys]
        nll = -probs_for_examples.log()
        loss = nll.mean()
        print(f'loss: {loss}')
        # Backward
        W.grad = None # Reset the gradient
        loss.backward()
        # Update
        W.data += -15.0*W.grad

train(1024)


## Conculsion of the NN approach
As seen above, the loss function (NLL) of the neural net approaches asymptotically the NLL of the "counting" approach (~2.45), so those two approaches end up in the same spot. Sweet!

### Parallels between P matrix and W matrix
The counting approach has P matrix with occurence counts/probabilites. This is equivalent to the output weights (e^weights more precisely, as those are log likelyhoods).
Smoothing the P array was achieved by adding a constant C to each cell between normalizing. The same thing can be achieved for NN by incentivizing weights W to be near zero - e.g. by choosing a different initialization than normal distribution. This is because e^0 = 1, so normalized it's a uniform distribution for each "row" of W.

You can incorporate this smoothiing into loss, by doing e.g.
`loss = nll.mean() + 0.01*(W**2).mean()`
This adds increases loss function for such composition of Ws that make them far from zero.

In [ ]:
g = torch.Generator().manual_seed(2147483647)
# Sampling from the trained NN
result = ''
input = 0 # Start character
while True:
    ienc = F.one_hot(torch.tensor([input]), num_classes=27).float()
    # Forward
    logits = ienc @ W
    # Softmax - transform the output of NN into distribution on probabilities
    counts = logits.exp()
    softmax = counts / counts.sum(1, keepdim=True)
    input = torch.multinomial(softmax, num_samples=1, generator=g).item()
    if input == 0: 
        break
    else: 
        result += itos[input]

print(result)